## EMPAQUETAR TODO EL PROCESO EN FUNCIONES

Estas funciones las iremos rescatando de los notebooks correspondientes que tendremos que ir a buscarlos

### Calidad de datos

In [1]:
#Traemos los pasos finales de calidad de datos trabajando siempre sobre una tabla temporal
def calidad_datos(x):
    
    #Modificar tipos
    temp = x.astype({'month': 'O', 'wday': 'O'})             
    
    #Imputar nulos
    temp.loc[x['event_name_1'].isna(),'event_name_1'] = 'Sin_evento'
    
    def imputar_moda(registros):
        #Calcula la moda del precio en ese producto
        moda = registros.sell_price.mode()[0]
        #Imputa los nulos
        registros.loc[registros.sell_price.isna(),'sell_price'] = moda
        #Devuelve todos los registros del producto
        return(registros)

    temp = temp.groupby('item_id', group_keys=False).apply(imputar_moda)
      
    return(temp)

### Creación de variables

In [2]:
#Creamos la función conjunta que contendrá todas las funciones individuales. Cuidado que va sobre 'x' y no sobre 'df'
def crear_variables(x):
    
    #DEMANDA INTERMITENTE
    #Función para localizar roturas de stock en los últimos 5 dias. Se puede cambiar n para cambiar la ventana móvil
    def rotura_stock(ventas, n = 5):
        cero_ventas = pd.Series(np.where(ventas == 0,1,0))
        num_ceros = cero_ventas.rolling(n).sum()
        rotura_stock = np.where(num_ceros == n,1,0)
        return(rotura_stock)
    
    #Muy importante ordenar correctamente los datos en función de lo que estemos buscando
    x = x.sort_values(by = ['store_id','item_id','date'])
    x['rotura_stock_3'] = x.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x, 3)).values
    x['rotura_stock_7'] = x.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,7)).values
    x['rotura_stock_15'] = x.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,15)).values
    
    
    #LAGS
    def crear_lags(x, variable, num_lags = 7):

        #Crea el objeto dataframe
        lags = pd.DataFrame()

        #Crea todos los lags
        for cada in range(1,num_lags+1):
            lags[variable + '_lag_'+ str(cada)] = x[variable].shift(cada)

        #Devuelve el dataframe de lags
        return(lags)
    
    lags_sell_price_df = (x.groupby(['store_id', 'item_id'])
                        .apply(lambda x: crear_lags(x = x, variable = 'sell_price', num_lags= 7))
                        .reset_index()
                        .set_index('date'))
    
    lags_rotura_stock_3_df = (x.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(x = x, variable = 'rotura_stock_3', num_lags= 1))
                            .reset_index()
                            .set_index('date'))
    
    lags_rotura_stock_7_df = (x.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(x = x, variable = 'rotura_stock_7', num_lags= 1))
                            .reset_index()
                            .set_index('date'))
    
    lags_rotura_stock_15_df = (x.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(x = x, variable = 'rotura_stock_15', num_lags= 1))
                            .reset_index()
                            .set_index('date'))
    
    lags_ventas_df = (x.groupby(['store_id','item_id'])
                    .apply(lambda x: crear_lags(x = x, variable = 'ventas', num_lags= 15))
                    .reset_index()
                    .set_index('date'))
    
    
    #VENTANAS MÓVILES
    def min_movil(x, variable, num_periodos = 7):
        minm = pd.DataFrame()
        for cada in range(2,num_periodos+1):
            minm[variable + '_minm_' + str(cada)] = x[variable].shift(1).rolling(cada).min()
        return(minm)
    
    def media_movil(x, variable, num_periodos = 7):
        mm = pd.DataFrame()
        for cada in range(2,num_periodos+1):
            mm[variable + '_mm_' + str(cada)] = x[variable].shift(1).rolling(cada).mean()
        return(mm)
    
    def max_movil(x, variable, num_periodos = 7):
        maxm = pd.DataFrame()
        for cada in range(2,num_periodos+1):
            maxm[variable + '_maxm_' + str(cada)] = x[variable].shift(1).rolling(cada).max()
        return(maxm)
    
    min_movil_df = (x.groupby(['store_id','item_id'])
                  .apply(lambda x: min_movil(x = x, variable = 'ventas', num_periodos= 15))
                  .reset_index()
                  .set_index('date'))
    
    media_movil_df = (x.groupby(['store_id','item_id'])
                    .apply(lambda x: media_movil(x = x, variable = 'ventas', num_periodos= 15))
                    .reset_index()
                    .set_index('date'))
    
    max_movil_df = (x.groupby(['store_id','item_id'])
                    .apply(lambda x: max_movil(x = x, variable = 'ventas', num_periodos= 15))
                    .reset_index()
                    .set_index('date'))
    
    
    #UNIR DATAFRAMES GENERADOS

    x_unido = pd.concat([x,
                      lags_sell_price_df,
                      lags_rotura_stock_3_df,
                      lags_rotura_stock_7_df,
                      lags_rotura_stock_15_df,
                      lags_ventas_df,
                      min_movil_df,
                      media_movil_df,
                      max_movil_df], axis = 1)

    # Eliminar columnas duplicadas
    x_unido = x_unido.loc[:,~x_unido.columns.duplicated()]
    
    x_unido.dropna(inplace=True)
    
    x_unido.drop(columns = ['sell_price','rotura_stock_3','rotura_stock_7','rotura_stock_15'],
                  inplace=True)
    
    #Crear una sola variable para el producto. Lo localizamos en la modelización
    x_unido.insert(loc=0,column='producto',value=x_unido.store_id + '_'+ x_unido.item_id)
    x_unido = x_unido.drop(columns = ['store_id','item_id'])
    
    return(x_unido)

### Transformación de variables

In [3]:
#Para poder usar esta función en otros proyectos con otras transformaciones habría que adaptarla según las necesidades
def transformar_variables(x,y=None,modo = 'entrenamiento'):
#Cuando queramos llamar la función en modo entrenamiento le pasaremos un 'x' y un 'y' y cuando la llamemos en modo ejecución
#le pasaremos un 'x' y el parámetro 'y' será el por defecto que es None
    '''
    He modificado esta función para que sirva tanto para entrenamiento como para ejecución:

    * Incluyendo el parámetro modo, que por defecto es entrenamiento
    * Haciendo que el parámetro 'y' sea opcional, ya que en ejecución no se usa

    Cuando se usa en modo entrenamiento aplica el método fit_transform y guarda los objetos.

    Mientras que cuando se usa en modo ejecución carga los objetos y aplica solo el método transform.
    '''    
    
    x.reset_index(inplace = True)

    #GESTION DE LOS ENCODERS
    nombre_ohe = 'ohe_retail.pickle'
    nombre_te = 'te_retail.pickle'
    ruta_ohe = ruta_proyecto + '/04_Modelos/' + nombre_ohe
    ruta_te = ruta_proyecto + '/04_Modelos/' + nombre_te
    
    #ONE HOT ENCODING
    var_ohe = ['event_name_1']
    if modo == 'entrenamiento':
        #Si está en entrenamiento aplica fit_transform y guarda el encoder
        ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')
        ohe_x = ohe.fit_transform(x[var_ohe])
        ohe_x = pd.DataFrame(ohe_x, columns = ohe.get_feature_names_out())
        with open(ruta_ohe, mode='wb') as file:
           pickle.dump(ohe, file)
    else:
        #Si está en ejecución recupera el guardado y solo aplica transform
        with open(ruta_ohe, mode='rb') as file:
            ohe = pickle.load(file)
        ohe_x = ohe.transform(x[var_ohe])
        ohe_x = pd.DataFrame(ohe_x, columns = ohe.get_feature_names_out())

    #TARGET ENCODING    
    var_te = ['month','wday','weekday']
    if modo == 'entrenamiento':
        #ASEGURAR QUE Y TIENE LOS MISMOS REGISTROS QUE X
        y.reset_index(inplace = True, drop = True)
        y = y.loc[y.index.isin(x.index)]
        #Si está en entrenamiento aplica fit_transform y guarda el encoder
        te = TargetEncoder(min_samples_leaf=100, return_df = False)
        te_x = te.fit_transform(x[var_te], y = y)
        nombres_te = [variable + '_te' for variable in var_te]
        te_x = pd.DataFrame(te_x, columns = nombres_te)
        with open(ruta_te, mode='wb') as file:
           pickle.dump(te, file)
    else:
        #Si está en ejecución recupera el guardado y solo aplica transform
        with open(ruta_te, mode='rb') as file:
            te = pickle.load(file)
        te_x = te.transform(x[var_te])
        nombres_te = [variable + '_te' for variable in var_te]
        te_x = pd.DataFrame(te_x, columns = nombres_te)
    
      
    #INTEGRAR, LIMPIAR Y DEVOLVER EL DATAFRAME
    #Eliminar las originales ya transformadas
    x = x.drop(columns=['event_name_1','month','wday','weekday'])
    #Incorporar los otros dataframes
    x = pd.concat([x,ohe_x,te_x], axis=1).set_index('date')

    #Salida
    return(x)

### Preselección de variables

In [4]:
#CUIDADO! Las funciones no son iguales que en los notebooks de desarrollo
def preseleccionar_variables(x,y):
    
    '''
    Sólo hay que usarla en entrenamiento.
    '''
    #ELIMINAR LA COLUMNA PRODUCTO Y EL INDEX
    x.reset_index(drop = True,inplace = True)
    x.drop(columns='producto',inplace = True)
    
    #ASEGURAR QUE 'Y' TIENE LOS MISMOS REGISTROS QUE 'X'
    y = y.loc[y.index.isin(x.index)]
    
    mutual_selector = mutual_info_regression(x,y)
    posicion_variable_limite = 82 #Debe coincidir con la preselección de variables que hayamos hecho
    ranking_mi = pd.DataFrame(mutual_selector, index = x.columns).reset_index()
    ranking_mi.columns = ['variable','importancia_mi']
    ranking_mi = ranking_mi.sort_values(by = 'importancia_mi', ascending = False)
    ranking_mi['ranking_mi'] = np.arange(0,ranking_mi.shape[0])
    entran_mi = ranking_mi.iloc[0:posicion_variable_limite].variable
    x_mi = x[entran_mi].copy()

    return(x_mi)

### Modelización

#### Entrenamiento

In [5]:
#
def modelizar(x_producto, y):
    #x_producto es el resultado de la combinación de los productos con cada tienda
    '''
    Esta función es la que hace la modelización individual.

    Recibe los datos de las x y la y de un producto.

    Encuentra los parámetros óptimos para ese producto.

    Devuelve el mejor modelo.
    '''
      
    #Excluye el producto como variable de modelización. Cuenta desde la posición 2 xq la posición 1 es la variable producto  
    var_modelizar = x_producto.columns.to_list()[2:]
    
    #Define la validación cruzada
    time_cv = TimeSeriesSplit(5, test_size = 8)
    
    #Define la parrilla de algoritmos
    pipe = Pipeline([('algoritmo',HistGradientBoostingRegressor())])
    
    grid = [ 
         {'algoritmo': [HistGradientBoostingRegressor()]
#          'algoritmo__learning_rate': [0.01,0.025,0.05,0.1],
#          'algoritmo__max_iter': [50,100,200],
#          'algoritmo__max_depth': [5,10,20,50],
#          'algoritmo__scoring': ['neg_mean_absolute_error'],
#          'algoritmo__l2_regularization': [0,0.25,0.5,0.75,1]
         }
                       
    ]
           
    #Crea los modelos
    random_search = RandomizedSearchCV(estimator = pipe,
                                   param_distributions = grid, 
                                   n_iter = 1, 
                                   cv = time_cv, 
                                   scoring = 'neg_mean_absolute_error', 
                                   verbose = 0,
                                   n_jobs = -1)
    
    modelo = random_search.fit(x_producto[var_modelizar],y)
    
    #Reentrena el mejor sobre todos los datos
    modelo_final = modelo.best_estimator_.fit(x_producto[var_modelizar],y)
    
    #Devuelve como salida el modelo final
    return(modelo_final)

In [6]:
def lanzar_entrenamiento(df):
    
    '''
    Esta función va recorriendo todos los productos y llamando a modelizar() para crear una lista total con todos los 
    modelos de todos los productos.

    Recibe el dataframe de las x ya limpio y segmentado por producto, y también la target.

    No devuelve nada, si no que guarda en disco el objeto ya entrenado con todos los modelos.
    '''
    
    lista_productos = list(df.producto.unique())
    
    lista_modelos = [] 
    
    for cada in lista_productos:
        
        #Renombra por claridad
        producto = cada
        target = 'ventas'

        x = df.loc[df.producto == producto].copy().drop(columns=target).copy()
        y = df.loc[df.producto == producto,'ventas'].copy()

        x = transformar_variables(x,y)
        x = preseleccionar_variables(x,y)
        
        #Llama a la funcion de modelizar
        modelo = modelizar(x,y)
        
        #Añade el modelo final a la lista
        lista_modelos.append((producto,modelo))
        
    #Guarda la lista de modelos entrenados
    nombre_modelos = 'lista_modelos_retail.pickle'
    ruta_modelos = ruta_proyecto + '/04_Modelos/' + nombre_modelos
    with open(ruta_modelos, mode='wb') as file:
       pickle.dump(lista_modelos, file)

#### Ejecución

In [7]:
#Lanzar la predicción
def lanzar_ejecucion(df):
    
    '''
    Esta función hace el forecast para cada producto, pero solo de un día.

    Recibe el nuevo dataset a predecir.
    
    Que tiene que tener la estructura del fichero DatosParaProduccion.csv de la carpeta Validación. Este dataset tiene la 
    cantidad de días necesario para hacer la predicción y las variables que hemos decidido que son necesarias
    
    Este sería el fichero que deberíamos pasarle al modelo para que realice las previsiones diarias cuando el sistema esté
    en producción

    Va recorriendo cada producto, cargando su modelo correspondiente, seleccionando sus datos, y haciendo las predicciones.

    Devuelve la predicción para todos los productos pero SOLO PARA EL DÍA QUE TOCA.
    
    Este sería el fichero que deberíamos pasarle al modelo para que realice las previsiones diarias cuando el modelo esté
    en producción
    '''
    
    #CARGA LOS MODELOS
    nombre_modelos = 'lista_modelos_retail.pickle'
    ruta_modelos = ruta_proyecto + '/04_Modelos/' + nombre_modelos
    with open(ruta_modelos, mode='rb') as file:
       lista_modelos = pickle.load(file)
    
    predicciones_df = pd.DataFrame(columns=['date','producto','ventas','prediccion'])
    
    for cada in range(0,len(lista_modelos)):

        producto = lista_modelos[cada][0]
        modelo = lista_modelos[cada][1]
        #Estas son las variables que usaremos de cada modelo en concreto que están guardadas en este atributo
        variables = modelo[0].feature_names_in_
        target = 'ventas'
        
        x = df.loc[df.producto == producto].copy().drop(columns=target).copy()
        y = df.loc[df.producto == producto,'ventas'].copy()
        
        date = df.reset_index().copy()
        date = date.loc[date.producto == producto,'date'].values

        #Transformacion de variables
        x = transformar_variables(x, modo = 'ejecucion')
        
        #Seleccion de variables con las que el modelo fue entrenado en la versión final. Debe ser exacto o no funcionará
        x = x[variables]
        
        #Cálculo de predicciones
        predicciones = pd.DataFrame(data={'date': date,
                                          'producto': producto,
                                          'ventas': y,
                                          'prediccion': modelo.predict(x)})

        predicciones['prediccion'] = predicciones.prediccion.astype('int')

        predicciones_df = pd.concat([predicciones_df,predicciones])
    
    predicciones_df = predicciones_df.loc[predicciones_df.index == predicciones_df.index.min()]
    return(predicciones_df)

In [8]:
def forecast_recursivo(x):
    
    '''
    Esta función es la que aplica el forecast recursivo para predecir los días que le definamos. En este caso 8 días
    
    Recibe el nuevo dataset a predecir que tiene que tener la estructura del fichero DatosParaProduccion.csv de la carpeta
    Validación. Ya que para aplicar la recursividad:

    * Va a predecir el primer día para el cual tenga toda la información (es decir 15 días hacia delante desde el día más 
    antiguo porque con los lags habremos provocado la generación de 15 días de nulos y no tendría toda la información)
    * Al finalizar graba la predicción de ventas en el registro a predecir y elimina los registros del día más antiguo del 
    dataframe. Esto será la predicción de 't'
    * En la siguiente iteración va a predecir el siguiente día (que será 't+1') y necesitará 15 días hacia atras que serán 
    14 días de datos reales + 1 día de predicción que lo tomaremos como dato real y así seguiremos con 't+1', 't+3',...

    Por ejemplo: Si el día más antiguo del dataset es el 09/12/2015 entonces el primer día que puede predecir (y del cual ya
    no tenemos dato) es el 24/12/2015.

    Cuando predice el dato del 24 para cada producto lo sobrescribe como sus ventas y elimina todos los registros del día 09

    Entonces el día más antiguo pasa a ser el día 10 y el día a predecir es el 25 y así hasta que finaliza 8 ciclos para 
    predecir la semana que queremos.
    '''
    #Deberemos poner el número de días que queremos que haga la predicción. En este caso 8 días
    for cada in range(0,8):
        paso1_df = calidad_datos(x)
        paso2_df = crear_variables(paso1_df)
        
        #Calcula la predicción
        f = lanzar_ejecucion(paso2_df)
        f['store_id'] = f.producto.str[:4]
        f['item_id'] = f.producto.str[5:]

        #Actualiza el dato de ventas con la predicción
        x.loc[(x.index.isin(f.date)) & (x.store_id.isin(f.store_id)) & (x.item_id.isin(f.item_id)),'ventas'] = f.prediccion
                                                              
        #Elimina el día más antiguo de x
        x = x.loc[x.index != x.index.min()]
        
    return(x)

## PROCESO

### REENTRENAMIENTO

In [12]:
import numpy as np
import pandas as pd
import pickle

#Estos paquetes serán diferentes en cada proyecto. Hay que poner los que necesitemo específicamente
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.pipeline import Pipeline

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Cargar los datos. Si necesitáramos reentrenar el modelo en el futuro deberíamos pasarle un histórico con similares características
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/002_RETAIL'
nombre_fichero_datos = 'trabajo.csv'
ruta_completa = ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,sep=',',parse_dates=['date'],index_col='date')

#Seleccionar solo las variables 'madre' que se han usado y que están en el excel
variables_finales = ['store_id',
                    'item_id',
                    'event_name_1',
                    'month',
                    'sell_price',
                    'ventas',
                    'wday',
                    'weekday',
                    'year']

df = df[variables_finales]

paso1_df = calidad_datos(df)
paso2_df = crear_variables(paso1_df)

lanzar_entrenamiento(paso2_df)

#No nos devuelve nada porque el objetivo de este proceso es entrenar el modelo y guardarlo en el disco

In [13]:
df

,store_id,item_id,event_name_1,month,sell_price,ventas,wday,weekday,year
date,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,NewYear,1,1.25,0,4,Tuesday,2013
2013-01-01,CA_3,FOODS_3_120,NewYear,1,4.98,0,4,Tuesday,2013
2013-01-01,CA_3,FOODS_3_202,NewYear,1,4.28,20,4,Tuesday,2013
2013-01-01,CA_3,FOODS_3_252,NewYear,1,1.48,34,4,Tuesday,2013
2013-01-01,CA_3,FOODS_3_288,NewYear,1,NaN,0,4,Tuesday,2013
...,...,...,...,...,...,...,...,...,...
2015-11-30,CA_4,FOODS_3_329,NaN,11,2.98,5,3,Monday,2015
2015-11-30,CA_4,FOODS_3_555,NaN,11,1.68,4,3,Monday,2015
2015-11-30,CA_4,FOODS_3_586,NaN,11,1.68,9,3,Monday,2015


### EVALUACIÓN

In [15]:
'''
Este código sirve para evaluar si el proceso está funcionando correctamente despues de hacer un reentrenamiento haciendo una
predicción de cada producto para un solo día con la función 'lanzar_ejecucion' y con el fichero 'validacion.csv'

Si quisieramos hacer una evaluación con los días que hemos marcado (En este caso 8 días), necesitamos usar la función de
'forecast_recursivo' y la estructura de los datos del fichero DatosParaProduccion.csv por lo que tendríamos que estructurar
el fichero de validacion.csv para que nos permitiera usarlo y después pasarle la función de 'forecast_recursivo'
'''

import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Cargar los datos
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/002_RETAIL'
nombre_fichero_datos = 'validacion.csv'
ruta_completa = ruta_proyecto + '/02_Datos/02_Validacion/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,sep=',',parse_dates=['date'],index_col='date')

#Seleccionar solo las variables 'madre' que se han usado y que están en el excel
variables_finales = ['store_id',
                    'item_id',
                    'event_name_1',
                    'month',
                    'sell_price',
                    'ventas',
                    'wday',
                    'weekday',
                    'year']


df = df[variables_finales]

paso1_df = calidad_datos(df)
paso2_df = crear_variables(paso1_df)

forecast_1dia = lanzar_ejecucion(paso2_df)

print('MAE es el error medio que está cometiendo sobre todos los productos')
print('MAE = ', mean_absolute_error(forecast_1dia.ventas,forecast_1dia.prediccion))

forecast_1dia
#Si saliera un valor negativo, sería un error porque no se pueden tener ventas negativas y el modelo no contempla devoluciones

MAE es el error medio que está cometiendo sobre todos los productos
MAE =  6.05


,date,producto,ventas,prediccion
2015-12-16,2015-12-16,CA_3_FOODS_3_090,0,-16
2015-12-16,2015-12-16,CA_3_FOODS_3_120,52,51
2015-12-16,2015-12-16,CA_3_FOODS_3_202,20,16
2015-12-16,2015-12-16,CA_3_FOODS_3_252,36,35
2015-12-16,2015-12-16,CA_3_FOODS_3_288,35,21
2015-12-16,2015-12-16,CA_3_FOODS_3_329,64,41
2015-12-16,2015-12-16,CA_3_FOODS_3_555,30,22
2015-12-16,2015-12-16,CA_3_FOODS_3_586,76,59
2015-12-16,2015-12-16,CA_3_FOODS_3_587,29,34
2015-12-16,2015-12-16,CA_3_FOODS_3_714,19,17


### EJECUCIÓN

In [18]:
'''
En principio, nosotros no tendremos el fichero 'DatosParaProduccion.csv', si no que se irá generando a partir de la info que
se vaya extayendo de la base de datos con un script de SQL que lo prepará otro equipo y que irá extrayendo diariamente la
información que le hayamos definido con las variables_finales que necesitemos y el histórico de días que hayamos definido.

A partir de ahí irán generando el fichero 'DatosParaProduccion.csv' con los datos actualizados y que nosotros rescataremos 
para usarlo y hacer las predicciones

De cara a construir el fichero, tenemos variables que usamos y variables que no usamos, pero debemos hay que incluir las 
variables porque se debe mantener la estructura de columnas necesaria

Las variables que NO usamos se pueden sustituir por ceros desde el día que queremos predecir porque no nos harán falta
Las variables que SI usaremos deberán rellenarse con las necesisades de información específicas que le hayamos definido al
equipo encargado de la extracción de datos de la base de datos

Ej. 
    * En el caso de 002_RETAIL store_id debe repetirse 10 veces cada uno porque tenemos 10 productos en cada tienda, por lo 
    que la fecha debe repetirse 20 verces porque tenemos dos tiendas gemelas y cada item_id aparecerá dos veces

    * En el caso del precio, lo normal es que haya un calendario de precios/promociones por lo que se podrá definir en el 
    momento o, en su defecto, le pondremos el último precio válido de ese producto

    * Dia de la semana y festivos se saca por cáculo y por calendario definido de festivos
    
    * El script de SQL le asignaría ceros a todos los valores a futuro porque no tiene información para rellenarlo y sería 
    nuestro código de Python el que accedería al fichero e iria sobreescribiendo esa información
'''
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.feature_selection import mutual_info_regression

from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

#Cargar los datos
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/002_RETAIL'
nombre_fichero_datos = 'DatosParaProduccion.csv'
ruta_completa = ruta_proyecto + '/02_Datos/02_Validacion/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,sep=';',parse_dates=['date'],index_col='date')

#Seleccionar solo las variables 'madre' que se han usado y que están en el excel
variables_finales = ['store_id',
                    'item_id',
                    'event_name_1',
                    'month',
                    'sell_price',
                    'ventas',
                    'wday',
                    'weekday',
                    'year']

df = df[variables_finales]

#Lanzar la predicción
forecast = forecast_recursivo(df)

pd.set_option('display.max_rows', None)
forecast.sort_values(by = ['store_id','item_id'])

,store_id,item_id,event_name_1,month,sell_price,ventas,wday,weekday,year
date,,,,,,,,,
17/12/2015,CA_3,FOODS_3_090,NaN,12,1.00,0,6,Thursday,2015
18/12/2015,CA_3,FOODS_3_090,NaN,12,1.00,1,7,Friday,2015
19/12/2015,CA_3,FOODS_3_090,NaN,12,1.00,0,1,Saturday,2015
20/12/2015,CA_3,FOODS_3_090,NaN,12,1.00,6,2,Sunday,2015
21/12/2015,CA_3,FOODS_3_090,NaN,12,1.00,4,3,Monday,2015
22/12/2015,CA_3,FOODS_3_090,NaN,12,1.00,5,4,Tuesday,2015
23/12/2015,CA_3,FOODS_3_090,NaN,12,1.00,2,5,Wednesday,2015
24/12/2015,CA_3,FOODS_3_090,Chanukah End,12,1.00,46,6,Thursday,0
25/12/2015,CA_3,FOODS_3_090,Christmas,12,1.00,17,7,Friday,0
